ksekinisame apo to baseline
meta skeftika na valo ta idia pou eixe gia abstracts kai gia tous authors
tora 8a prospa8iso na kano hyperparameter tuning kai na xrisimopoiiso ki alla modela
Meta 8a prospa8iso na xrisimopoiiso cosine similrity me tin xrisi to word2vec
to cosine similarity den doulevei giati ta vector exoun diaforetika sized kati prepei na allakso



In [10]:
import networkx as nx
import gensim
import csv
import numpy as np
from random import randint
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
import nltk
from gensim.models import Word2Vec
from gensim.models import Doc2Vec
from collections import Counter
from nltk.corpus import stopwords

In [11]:
# Create a graph
G = nx.read_edgelist('edgelist.txt', delimiter=',', create_using=nx.Graph(), nodetype=int)
nodes = list(G.nodes())
n = G.number_of_nodes()
m = G.number_of_edges()
print('Number of nodes:', n)
print('Number of edges:', m)

Number of nodes: 138499
Number of edges: 1091955


In [12]:
# Read the abstract of each paper
abstracts = dict()
with open('abstracts.txt', 'r', encoding='utf-8') as f:
    for line in f:
        node, abstract = line.split('|--|')
        abstracts[int(node)] = abstract

In [13]:
# Read the authors of each paper
authors = dict()
with open('authors.txt', 'r', encoding='utf-8') as f:
    for line in f:
        node, author = line.split('|--|')
        authors[int(node)] = author

In [14]:
cluster = nx.clustering(G)

In [15]:
tokenized_authors = dict()
for node in authors:
    for word in authors[node].split(','):
        tokenized_authors[node] = [word for word in word_tokenize(word)]

In [16]:
stop_words = set(stopwords.words('english'))
stop_words.update(['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}'])

In [17]:
tokenized_abstracts = dict()
for node in abstracts:
    tokenized_abstracts[node] = []
    for sent in sent_tokenize(abstracts[node]):
        for i in word_tokenize(sent):
            if i.lower() in stop_words:
                continue
            else:
                tokenized_abstracts[node].append(i.lower())

In [18]:
# Map text to set of terms
for node in abstracts:
    abstracts[node] = set(abstracts[node].split(','))

In [19]:
# Map authors to set of terms
for node in authors:
    authors[node] = set(authors[node].split(','))

In [20]:
import math
epsilon = 1e-6
def counter_cosine_similarity(c1, c2):
    terms = set(c1).union(c2)
    dotprod = sum(c1.get(k, 0) * c2.get(k, 0) for k in terms)
    magA = math.sqrt(sum(c1.get(k, 0)**2 for k in terms))
    magB = math.sqrt(sum(c2.get(k, 0)**2 for k in terms))
    return dotprod / ((magA * magB) + epsilon)

In [21]:
#import gensim.models.keyedvectors as word2vec
#model = word2vec.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
#model2 = Word2Vec(tokenized_abstracts.values(), window=5, min_count=1, workers=4)
from sklearn.cluster import KMeans
data = list(tokenized_abstracts.values())
kmeans = KMeans(n_clusters=5, random_state=0).fit(data)
kmeans

In [24]:
rank = nx.pagerank(G)

In [ ]:
data = tokenized_abstracts.values()
data

In [38]:
#import countvectorizer and tfidfvectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(tokenizer=word_tokenize, stop_words=stop_words)


In [35]:
Counter(tokenized_abstracts[0])

Counter({'development': 1,
         'automated': 1,
         'system': 3,
         'quality': 2,
         'assessment': 1,
         'aerodrome': 2,
         'ground': 1,
         'lighting': 1,
         'agl': 4,
         'accordance': 1,
         'associated': 1,
         'standards': 2,
         'recommendations': 1,
         'presented': 2,
         'composed': 1,
         'image': 4,
         'sensor': 1,
         'placed': 1,
         'inside': 1,
         'cockpit': 1,
         'aircraft': 1,
         'record': 1,
         'images': 1,
         'normal': 1,
         'descent': 1,
         'model-based': 1,
         'methodology': 1,
         'used': 2,
         'ascertain': 1,
         'optimum': 1,
         'match': 1,
         'template': 1,
         'actual': 1,
         'data': 3,
         'order': 1,
         'calculate': 1,
         'position': 2,
         'orientation': 2,
         'camera': 2,
         'instant': 1,
         'acquired': 1,
         'along': 1,
         'p

In [25]:
# its class label is 1 if it corresponds to an edge and 0, otherwise.
# Use the following 3 features for each pair of nodes:
# (1) sum of number of unique terms of the two nodes' abstracts
# (2) absolute value of difference of number of unique terms of the two nodes' abstracts
# (3) number of common terms between the abstracts of the two nodes
# (4) sum of number of unique terms of the two nodes' authors
# (5) absolute value of difference of number of unique terms of the two nodes' authors
# (6) 

X_train = np.zeros((2*m, 12))
y_train = np.zeros(2*m)
n = G.number_of_nodes()
for i,edge in enumerate(G.edges()):
    # an edge
    X_train[i,0] = len(abstracts[edge[0]]) + len(abstracts[edge[1]])
    X_train[i,1] = abs(len(abstracts[edge[0]]) - len(abstracts[edge[1]]))
    X_train[i,2] = len(abstracts[edge[0]].intersection(abstracts[edge[1]]))
    X_train[i,3] = len(authors[edge[0]]) + len(authors[edge[1]])
    X_train[i,4] = abs(len(authors[edge[0]]) - len(authors[edge[1]]))
    X_train[i,5] = len(authors[edge[0]].intersection(authors[edge[1]]))
    X_train[i,6] = counter_cosine_similarity(Counter(tokenized_authors[edge[0]]), Counter(tokenized_authors[edge[1]]))
    X_train[i,7] = counter_cosine_similarity(Counter(tokenized_abstracts[edge[0]]), Counter(tokenized_abstracts[edge[1]]))
    X_train[i,8] = rank[edge[0]] + rank[edge[1]]
    X_train[i,9] = abs(rank[edge[0]] - rank[edge[1]])
    X_train[i,10] = cluster[edge[0]] + cluster[edge[1]]
    X_train[i,11] = abs(cluster[edge[0]] - cluster[edge[1]])
    y_train[i] = 1

    # a randomly generated pair of nodes
    n1 = randint(0, n-1)
    n2 = randint(0, n-1)
    X_train[m+i,0] = len(abstracts[n1]) + len(abstracts[n2])
    X_train[m+i,1] = abs(len(abstracts[n1]) - len(abstracts[n2]))
    X_train[m+i,2] = len(abstracts[n1].intersection(abstracts[n2]))
    X_train[m+i,3] = len(authors[n1]) + len(authors[n2])
    X_train[m+i,4] = abs(len(authors[n1]) - len(authors[n2]))
    X_train[m+i,5] = len(authors[n1].intersection(authors[n2]))
    X_train[m+i,6] = counter_cosine_similarity(Counter(tokenized_authors[n1]), Counter(tokenized_authors[n2]))
    X_train[m+i,7] = counter_cosine_similarity(Counter(tokenized_abstracts[n1]), Counter(tokenized_abstracts[n2]))
    X_train[m+i,8] = rank[n1] + rank[n2] # sum of ranks of the two nodes
    X_train[m+i,9] = abs(rank[n1] - rank[n2]) # absolute value of difference of ranks of the two nodes
    X_train[m+i,10] = cluster[n1] + cluster[n2] # sum of clusters of the two nodes
    X_train[m+i,11] = abs(cluster[n1] - cluster[n2]) # absolute value of difference of clusters of the two nodes
    y_train[m+i] = 0

In [26]:
print('Size of training matrix:', X_train.shape)

Size of training matrix: (2183910, 12)


In [27]:
# Read test data. Each sample is a pair of nodes
node_pairs = list()
with open('test.txt', 'r') as f:
    for line in f:
        t = line.split(',')
        node_pairs.append((int(t[0]), int(t[1])))

In [28]:
# Create the test matrix. Use the same 4 features as above
X_test = np.zeros((len(node_pairs), 12))
for i,node_pair in enumerate(node_pairs):
    X_test[i,0] = len(abstracts[node_pair[0]]) + len(abstracts[node_pair[1]])
    X_test[i,1] = abs(len(abstracts[node_pair[0]]) - len(abstracts[node_pair[1]]))
    X_test[i,2] = len(abstracts[node_pair[0]].intersection(abstracts[node_pair[1]]))
    X_test[i,3] = len(authors[node_pair[0]]) + len(authors[node_pair[1]])
    X_test[i,4] = abs(len(authors[node_pair[0]]) - len(authors[node_pair[1]]))
    X_test[i,5] = len(authors[node_pair[0]].intersection(authors[node_pair[1]]))
    X_test[i,6] = counter_cosine_similarity(Counter(tokenized_authors[node_pair[0]]), Counter(tokenized_authors[node_pair[1]]))
    X_test[i,7] = counter_cosine_similarity(Counter(tokenized_abstracts[node_pair[0]]), Counter(tokenized_abstracts[node_pair[1]]))
    X_test[i,8] = rank[node_pair[0]] + rank[node_pair[1]]
    X_test[i,9] = abs(rank[node_pair[0]] - rank[node_pair[1]])
    X_test[i,10] = cluster[node_pair[0]] + cluster[node_pair[1]]
    X_test[i,11] = abs(cluster[node_pair[0]] - cluster[node_pair[1]])
    
print('Size of test matrix:', X_test.shape)

Size of test matrix: (106692, 12)


In [29]:
X_train, y_train = shuffle(X_train, y_train)

In [33]:
# Use logistic regression to predict if two nodes are linked by an edge
logreg = LogisticRegression(C=1e3, solver='liblinear')
logreg.fit(X_train, y_train)


y_pred = logreg.predict_proba(X_test)
y_pred = y_pred[:,1]

In [34]:
# Write predictions to a file
predictions = zip(range(len(y_pred)), y_pred)
with open("submissions2.csv","w") as pred:
    csv_out = csv.writer(pred)
    csv_out.writerow(['id','predicted'])
    for row in predictions:
        csv_out.writerow(row)